# Chapter 1: ETF Data Collection

## This Chapter
We build and maintain our ETF database:
1. Connect to DEGIRO
2. Apply base filter criteria (Irish domicile, accumulating, EUR)
3. Fetch price data for each ETF from JustETF
4. Save everything to SQLite database for easy updates

**Note**: We collect broadly here (no fund size or history filters). More restrictive filters are applied later when using the data for analysis.

## Output
- `data/etf_database.db` - SQLite database containing:
  - `etfs` table: ETF metadata (name, TER, fund size, etc.)
  - `prices` table: Historical price data

## Monthly Updates
After initial setup, run Section 1.5 to update prices for existing ETFs and add any new ones.

## 1.1 Setup & Connect to DEGIRO

In [1]:
# Imports
import sys
import time
import warnings
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
from tqdm import tqdm
import justetf_scraping

# Add current directory to path
sys.path.insert(0, str(Path.cwd()))

warnings.filterwarnings('ignore')

In [2]:
# Connect to DEGIRO
from degiro_client import get_client

client = get_client()
api = client.api

print("Connected to DEGIRO")

DEGIRO: Connecting...
  ACTION REQUIRED:
  Open the DEGIRO app on your phone
  Tap 'Yes' to approve this login
  Waiting for approval...

Connected successfully!
Connected to DEGIRO


In [3]:
# Initialize database (creates if not exists, connects if exists)
from etf_database import ETFDatabase

db = ETFDatabase("data/etf_database.db")

stats = db.get_stats()
if stats['total_etfs'] > 0:
    print(f"Connected to existing database: {stats['total_etfs']} ETFs, {stats['total_price_records']:,} prices")
else:
    print("Created new empty database")

Created new empty database


## 1.2 Fetch ETF Universe from DEGIRO

In [5]:
from etf_fetcher import ETFFetcher, ETFFilter

# Initialize fetcher (uses existing DEGIRO connection)
fetcher = ETFFetcher(verbose=True)

In [6]:
# Define filter criteria
satellite_filter = ETFFilter(
    isin_prefix="IE00",           # Irish domiciled (tax efficient)
    distribution="Accumulating",  # Reinvest dividends
    currency="EUR",               # Currency in Euros
)

In [7]:
# Fetch ETFs matching our criteria
df_universe = fetcher.fetch(satellite_filter)
print(f"\nFound {len(df_universe)} ETFs matching criteria")

Fetching DEGIRO ETF catalog...


  Fetching: 100%|███████████████████████| 7742/7742 [00:43<00:00, 174.37 ETFs/s]


  Found 7565 ETFs on DEGIRO
Loading JustETF data...
  Loaded 3926 ETFs from JustETF

Filtering 2806 ETFs...
  Domicile (IE00): 1709 ETFs
  JustETF match: 1636 ETFs
  Distribution (Accumulating): 1104 ETFs
  Currency (EUR): 937 ETFs
  Fetching data history from JustETF...


  Checking history: 100%|█████████████████████| 902/902 [08:39<00:00,  1.74it/s]

  Deduplicated: 902 ETFs (by longest history)

Result: 902 ETFs

Found 902 ETFs matching criteria


In [8]:
# Filter out leveraged ETFs
leverage_keywords = ['leveraged', 'leverage', '2x', '3x', 'ultra', 'double', 'triple',
                     'levered', 'geared', 'x2', 'x3', '200%', '300%']

initial_count = len(df_universe)
df_universe = df_universe[
    ~df_universe['Name'].str.lower().str.contains('|'.join(leverage_keywords), na=False)
]

filtered_count = initial_count - len(df_universe)
print(f"Filtered out {filtered_count} leveraged ETFs")
print(f"Remaining ETFs: {len(df_universe)}")

Filtered out 30 leveraged ETFs
Remaining ETFs: 872


## 1.3 Add ETFs to Database

In [9]:
# Add all ETFs to database
# Helper to convert pandas NA to None for SQLite
def to_python(val):
    """Convert pandas NA/NaN to Python None."""
    if pd.isna(val):
        return None
    return val

added_count = 0
updated_count = 0

for _, row in tqdm(df_universe.iterrows(), total=len(df_universe), desc="Adding ETFs to database"):
    is_new = db.add_etf(
        isin=row['ISIN'],
        name=row['Name'],
        vwd_id=str(row.get('vwdId', '')),
        exchange=to_python(row.get('exchange')),
        currency=to_python(row.get('currency')),
        ter=to_python(row.get('TER')),
        fund_size=to_python(row.get('fund_size')),
        distribution='Accumulating',
        months_of_data=to_python(row.get('months_of_data'))
    )
    if is_new:
        added_count += 1
    else:
        updated_count += 1

print(f"\nAdded {added_count} new ETFs")
print(f"Updated {updated_count} existing ETFs")

Adding ETFs to database: 100%|██████████| 872/872 [00:14<00:00, 59.41it/s] 


Added 872 new ETFs
Updated 0 existing ETFs


## 1.4 Fetch and Store Price Data

In [10]:
# Fetch price data from JustETF and store in database
# This fetches FULL history for each ETF and validates overlap before replacing

isins = df_universe['ISIN'].tolist()
success_count = 0
fail_count = 0
validation_warnings = []

print(f"Fetching price data for {len(isins)} ETFs...")
print("(This may take several minutes)\n")

for isin in tqdm(isins, desc="Fetching prices"):
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            prices = justetf_scraping.load_chart(isin, currency='EUR')
        
        if prices is not None and len(prices) > 0:
            # Convert to Series if DataFrame
            if isinstance(prices, pd.DataFrame):
                prices = prices.iloc[:, 0]
            
            # Validate new prices against existing (if any)
            validation = db.validate_new_prices(isin, prices)
            
            if not validation['valid']:
                # Price mismatch in overlap period - warn but continue
                validation_warnings.append({
                    'isin': isin,
                    'mismatches': len(validation['mismatches']),
                    'overlap_days': validation['overlap_days'],
                    'sample': validation['mismatches'][:3]  # First 3 mismatches
                })
            
            # Store in database (replace=True for full refresh)
            records_added = db.update_prices(isin, prices, replace=True)
            if records_added > 0:
                success_count += 1
            else:
                fail_count += 1
        else:
            fail_count += 1
    except Exception as e:
        fail_count += 1
    
    # Rate limiting for JustETF
    time.sleep(0.3)

print(f"\nSuccessfully fetched prices for {success_count}/{len(isins)} ETFs")
if fail_count > 0:
    print(f"Failed to fetch prices for {fail_count} ETFs")

# Show validation warnings (if any)
if validation_warnings:
    print(f"\n{'='*70}")
    print(f"WARNING: {len(validation_warnings)} ETFs had price mismatches in overlap period")
    print(f"{'='*70}")
    print("(Historical prices may have been corrected by the data source)\n")
    
    for warn in validation_warnings[:10]:  # Show first 10
        print(f"  {warn['isin']}: {warn['mismatches']} mismatches in {warn['overlap_days']} overlap days")
        if warn['sample']:
            for date, old_p, new_p in warn['sample'][:2]:
                pct_diff = (new_p - old_p) / old_p * 100
                print(f"    - {date.strftime('%Y-%m-%d')}: {old_p:.2f} -> {new_p:.2f} ({pct_diff:+.2f}%)")
    
    if len(validation_warnings) > 10:
        print(f"\n  ... and {len(validation_warnings) - 10} more ETFs with warnings")
    
    print(f"\n{'='*70}")

Fetching price data for 872 ETFs...
(This may take several minutes)



Fetching prices: 100%|██████████| 872/872 [09:52<00:00,  1.47it/s]


Successfully fetched prices for 843/872 ETFs
Failed to fetch prices for 29 ETFs


In [11]:
# Data Quality Check
# Verify price data completeness and identify any gaps

print("="*70)
print("DATA QUALITY CHECK")
print("="*70)

all_isins = db.list_isins()
quality_issues = []
good_etfs = 0

for isin in tqdm(all_isins, desc="Checking data quality"):
    prices = db.load_prices(isin)
    etf_info = db.get_etf(isin)
    name = etf_info['name'][:40] if etf_info else isin
    
    if len(prices) == 0:
        quality_issues.append((isin, name, "NO DATA", 0, None))
        continue
    
    # Calculate metrics
    date_range = (prices.index.max() - prices.index.min()).days
    expected_trading_days = date_range * 5 / 7  # Rough estimate
    actual_days = len(prices)
    coverage = actual_days / expected_trading_days if expected_trading_days > 0 else 0
    
    # Check for gaps (more than 5 consecutive missing days)
    date_diff = prices.index.to_series().diff().dt.days
    max_gap = date_diff.max() if len(date_diff) > 0 else 0
    
    # Check for stale data (last update more than 7 days ago)
    days_since_update = (pd.Timestamp.now() - prices.index.max()).days
    
    # Flag issues
    issues = []
    if coverage < 0.85:
        issues.append(f"Low coverage ({coverage:.0%})")
    if max_gap > 10:
        issues.append(f"Gap of {max_gap} days")
    if days_since_update > 7:
        issues.append(f"Stale ({days_since_update}d old)")
    if len(prices) < 252:  # Less than 1 year of data
        issues.append(f"Short history ({len(prices)} days)")
    
    if issues:
        quality_issues.append((isin, name, ", ".join(issues), len(prices), prices.index.max()))
    else:
        good_etfs += 1

# Summary
print(f"\n{'Status':<15} {'Count':>8}")
print("-"*25)
print(f"{'Good ETFs':<15} {good_etfs:>8}")
print(f"{'With Issues':<15} {len(quality_issues):>8}")
print(f"{'Total':<15} {len(all_isins):>8}")

# Show issues if any
if quality_issues:
    print(f"\n{'='*70}")
    print("ETFs WITH DATA QUALITY ISSUES:")
    print(f"{'='*70}")
    print(f"{'ISIN':<15} {'Name':<40} {'Issue':<30}")
    print("-"*85)
    for isin, name, issue, days, last_date in sorted(quality_issues, key=lambda x: x[2]):
        print(f"{isin:<15} {name:<40} {issue:<30}")
    
    # Categorize issues
    no_data = [q for q in quality_issues if "NO DATA" in q[2]]
    stale = [q for q in quality_issues if "Stale" in q[2]]
    gaps = [q for q in quality_issues if "Gap" in q[2]]
    
    print(f"\nIssue Summary:")
    if no_data:
        print(f"  - No data: {len(no_data)} ETFs")
    if stale:
        print(f"  - Stale data: {len(stale)} ETFs")
    if gaps:
        print(f"  - Large gaps: {len(gaps)} ETFs")

print(f"\n{'='*70}")
print(f"Data quality check complete. {good_etfs}/{len(all_isins)} ETFs have good data.")
print(f"{'='*70}")

DATA QUALITY CHECK


Checking data quality: 100%|██████████| 872/872 [00:04<00:00, 187.77it/s]


Status             Count
-------------------------
Good ETFs            818
With Issues           54
Total                872

ETFs WITH DATA QUALITY ISSUES:
ISIN            Name                                     Issue                         
-------------------------------------------------------------------------------------
IE00000EF730    iShares Europe Equity Enhanced Active UC NO DATA                       
IE00002ZKAP0    Xtrackers Europe Equity Enhanced Active  NO DATA                       
IE00004PGEY9    JPM Eurozone Research Enhanced Index Equ NO DATA                       
IE00004S2680    Vanguard EUR Eurozone Government 1-3 Yea NO DATA                       
IE0000902GT6    WisdomTree Megatrends UCITS ETF - USD Ac NO DATA                       
IE0000EAPBT6    JPMorgan US Equity Premium Income Active NO DATA                       
IE0000FCGYF9    Invesco S&P China A MidCap 500 Swap UCIT NO DATA                       
IE0000H445G8    VanEck New China ESG UCITS ETF     

In [12]:
# Retry fetching prices for ETFs with no data
# (Rate limiting may have caused failures during initial fetch)

# Find ETFs with no price data
no_data_isins = [isin for isin, _, issue, _, _ in quality_issues if "NO DATA" in issue]

if no_data_isins:
    print(f"Retrying {len(no_data_isins)} ETFs with no data...")
    print("(Using longer delay to avoid rate limiting)\n")
    
    retry_success = 0
    retry_fail = 0
    
    for isin in tqdm(no_data_isins, desc="Retrying"):
        try:
            time.sleep(1.0)  # Longer delay for retry
            
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                prices = justetf_scraping.load_chart(isin, currency='EUR')
            
            if prices is not None and len(prices) > 0:
                if isinstance(prices, pd.DataFrame):
                    prices = prices.iloc[:, 0]
                
                records_added = db.update_prices(isin, prices, replace=True)
                if records_added > 0:
                    retry_success += 1
                else:
                    retry_fail += 1
            else:
                retry_fail += 1
        except Exception as e:
            retry_fail += 1
    
    print(f"\nRetry results: {retry_success} succeeded, {retry_fail} still failed")
    
    # Show updated stats
    stats = db.get_stats()
    print(f"ETFs with prices: {stats['etfs_with_prices']}/{stats['total_etfs']}")
else:
    print("No ETFs need retry - all have price data!")

Retrying 29 ETFs with no data...
(Using longer delay to avoid rate limiting)



Retrying: 100%|██████████| 29/29 [00:38<00:00,  1.33s/it]



Retry results: 29 succeeded, 0 still failed
ETFs with prices: 872/872


In [13]:
# Database summary
stats = db.get_stats()

print("="*60)
print("DATABASE SUMMARY")
print("="*60)
print(f"Total ETFs:          {stats['total_etfs']}")
print(f"ETFs with prices:    {stats['etfs_with_prices']}")
print(f"Total price records: {stats['total_price_records']:,}")
if stats['price_date_range']:
    print(f"Price date range:    {stats['price_date_range'][0]} to {stats['price_date_range'][1]}")
print(f"Database size:       {stats['db_size_mb']:.2f} MB")
print("="*60)

DATABASE SUMMARY
Total ETFs:          872
ETFs with prices:    872
Total price records: 2,063,930
Price date range:    2005-11-18 to 2025-12-12
Database size:       241.35 MB


## 1.5 Monthly Workflow

**Each month, follow these steps:**

1. **First**: Run the cell below to save a metadata snapshot (preserves current TER, fund_size values in history)
2. **Then**: Re-run cells 1.1 through 1.4 to refresh everything

**What happens when you re-run:**
- ✅ New ETFs matching criteria are added to the database
- ✅ Existing ETF metadata (TER, fund_size, months_of_data) is updated with latest values
- ✅ All price data is refreshed (validated against existing data, then replaced)
- ✅ The metadata snapshot preserves historical values for tracking changes over time

In [14]:
# Save metadata snapshot BEFORE re-running the fetch
# This preserves current fund_size, TER values in history

snapshot_count = db.save_metadata_snapshot()
print(f"Saved metadata snapshot for {snapshot_count} ETFs")

Saved metadata snapshot for 872 ETFs


In [15]:
# View metadata history (tracks fund_size, TER changes over time)
history = db.load_metadata_history()
if len(history) > 0:
    print(f"Metadata history: {len(history)} records across {history['isin'].nunique()} ETFs")
    snapshots = history['snapshot_date'].nunique()
    print(f"Snapshots: {snapshots} dates")
    print(f"Date range: {history['snapshot_date'].min().date()} to {history['snapshot_date'].max().date()}")
else:
    print("No metadata history yet. Run the snapshot cell above before re-fetching.")

Metadata history: 872 records across 872 ETFs
Snapshots: 1 dates
Date range: 2025-12-15 to 2025-12-15


## 1.6 Quick Data Access Examples

In [16]:
# Example: Load ETF universe
universe = db.load_universe()
print(f"ETF Universe: {len(universe)} ETFs")
print(universe[['isin', 'name', 'ter', 'fund_size']].head(10))

ETF Universe: 872 ETFs
           isin                                               name   ter  \
0  IE000K1P4V37  AMUNDI MSCI World SRI Climate Net Zero Ambitio...  0.20   
1  IE0003A512E4  ARK Artificial Intelligence & Robotics UCITS E...  0.75   
2  IE000O5M6XO1           ARK Genomic Revolution UCITS ETF USD Acc  0.75   
3  IE000GA3D489               ARK Innovation UCITS ETF USD Acc ETF  0.75   
4  IE000AON7ET1  ARK Space & Defence Innovation UCITS ETF (ARKX...  0.75   
5  IE0003IT72N9  AXA IM ACT Biodiversity Equity UCITS ETF EUR H...  0.53   
6  IE000SBHVL31   AXA IM ACT Biodiversity Equity UCITS ETF USD Acc  0.50   
7  IE000Z8BHG02                AXA IM ACT Climate Equity UCITS ETF  0.50   
8  IE000E66LX20    AXA IM ACT Climate Equity UCITS ETF EUR Hdg Acc  0.53   
9  IE000GLIXPP3  AXA IM MSCI Emerging Markets Equity PAB UCITS ETF  0.24   

   fund_size  
0      350.0  
1      301.0  
2       40.0  
3      309.0  
4        2.0  
5      116.0  
6       13.0  
7        7.0  
8    

In [18]:
# Summary
print("="*60)
print("CHAPTER 1 COMPLETE - ETF Data Collection")
print("="*60)
stats = db.get_stats()
print(f"\nDatabase: data/etf_database.db")
print(f"Total ETFs: {stats['total_etfs']}")
print(f"ETFs with prices: {stats['etfs_with_prices']}")
print(f"Total price records: {stats['total_price_records']:,}")
print(f"Database size: {stats['db_size_mb']:.2f} MB")
print(f"\nBase filter criteria:")
print(f"  - Irish domicile (IE00)")
print(f"  - Accumulating distribution")
print(f"  - EUR currency")
print(f"  - No leveraged ETFs")
print(f"\nMonthly workflow:")
print(f"  1. Run Section 1.5 first (save metadata snapshot)")
print(f"  2. Re-run cells 1.1 through 1.4")
print(f"  3. Check Section 1.5 to view metadata history")
print("="*60)

CHAPTER 1 COMPLETE - ETF Data Collection

Database: data/etf_database.db
Total ETFs: 872
ETFs with prices: 872
Total price records: 2,063,930
Database size: 241.44 MB

Base filter criteria:
  - Irish domicile (IE00)
  - Accumulating distribution
  - EUR currency
  - No leveraged ETFs

Monthly workflow:
  1. Run Section 1.5 first (save metadata snapshot)
  2. Re-run cells 1.1 through 1.4
  3. Check Section 1.5 to view metadata history
